# Tutorial 6: Caso de Estudio End-to-End (Versión Interactiva)

## Detección de Crisis Epilépticas con TDA

**Autor:** MARK-126  
**Nivel:** Avanzado  
**Tiempo estimado:** 180-240 minutos

---

## 🎯 Objetivos del Caso de Estudio

1. ✅ Trabajar con datos reales de EEG clínico
2. ✅ Pipeline completo de preprocesamiento profesional
3. ✅ Aplicar TDA a problema médico real
4. ✅ Construir clasificador con características topológicas
5. ✅ Evaluar con métricas clínicas rigurosas
6. ✅ Interpretar resultados neurobiológicamente

---

## ⚠️ Nota sobre Ejercicios

Este notebook contiene **3 ejercicios interactivos avanzados** del pipeline completo.

---

<a name='toc'></a>
## 📚 Tabla de Contenidos

- [1 - Setup e Importaciones](#1)
- [2 - Generación de Datos EEG](#2)
- [3 - Preprocesamiento](#3)
    - [Ejercicio 1 - preprocess_eeg](#ex-1)
- [4 - Pipeline TDA](#4)
- [5 - Extracción de Características](#5)
    - [Ejercicio 2 - extract_comprehensive_features](#ex-2)
- [6 - Machine Learning](#6)
    - [Ejercicio 3 - train_topological_classifier](#ex-3)
- [7 - Evaluación y Resumen](#7)

---

<a name='1'></a>
## 1 - Setup e Importaciones

[Volver al índice](#toc)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from ripser import ripser
from scipy import signal
from scipy.stats import zscore, poisson
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import pandas as pd

from tda_tests import (
    test_preprocess_eeg_tutorial6,
    test_extract_comprehensive_features_tutorial6,
    test_train_topological_classifier
)

plt.style.use('seaborn-v0_8-whitegrid')
np.random.seed(42)
print("✅ Setup completado")

<a name='2'></a>
## 2 - Generación de Datos EEG Sintéticos

[Volver al índice](#toc)

Generamos EEG que simula estados ictal (crisis) e interictal (normal).

In [ ]:
def generate_realistic_eeg_segment(duration=10, fs=256, state='interictal', n_channels=23):
    n_samples = int(duration * fs)
    t = np.linspace(0, duration, n_samples)
    eeg_data = np.zeros((n_channels, n_samples))
    
    for ch in range(n_channels):
        if state == 'interictal':
            alpha = 0.3 * np.sin(2 * np.pi * np.random.uniform(8, 13) * t)
            beta = 0.2 * np.sin(2 * np.pi * np.random.uniform(13, 30) * t)
            theta = 0.15 * np.sin(2 * np.pi * np.random.uniform(4, 8) * t)
            noise = 0.3 * np.random.randn(n_samples)
            eeg_data[ch] = alpha + beta + theta + noise
        elif state == 'ictal':
            seizure_freq = np.random.uniform(3, 5)
            spike_wave = 2.5 * np.sin(2 * np.pi * seizure_freq * t)
            harmonics = 0.8 * np.sin(2 * np.pi * 2 * seizure_freq * t)
            harmonics += 0.4 * np.sin(2 * np.pi * 3 * seizure_freq * t)
            hfo = 0.3 * np.sin(2 * np.pi * np.random.uniform(80, 200) * t)
            phase_offset = np.random.uniform(0, 0.2)
            noise = 0.15 * np.random.randn(n_samples)
            eeg_data[ch] = spike_wave + harmonics + hfo + noise
            eeg_data[ch] = np.roll(eeg_data[ch], int(phase_offset * fs))
    
    return eeg_data, t

print("🧠 Generando datos EEG...\n")
eeg_interictal, t = generate_realistic_eeg_segment(duration=10, state='interictal')
eeg_ictal, _ = generate_realistic_eeg_segment(duration=10, state='ictal')
print(f"✅ Shape: {eeg_interictal.shape}")

<a name='3'></a>
## 3 - Preprocesamiento Profesional

[Volver al índice](#toc)

<a name='ex-1'></a>
### Ejercicio 1 - preprocess_eeg

Implementa el pipeline de preprocesamiento profesional para EEG clínico.

**Pasos:**
1. Filtro bandpass (0.5-50 Hz)
2. Notch filter (60 Hz)
3. Common average reference (CAR)
4. Normalización z-score por canal

In [ ]:
# EJERCICIO 1: Preprocesar EEG

def preprocess_eeg(eeg_data, fs=256):
    """
    Pipeline de preprocesamiento profesional para EEG.
    
    Arguments:
    eeg_data -- array (n_channels, n_samples)
    fs -- frecuencia de muestreo (Hz)
    
    Returns:
    eeg_normalized -- EEG preprocesado
    """
    n_channels, n_samples = eeg_data.shape
    
    # 1. Filtro bandpass (0.5-50 Hz)
    # Usar scipy.signal.butter y filtfilt
    # (approx. 5 lines)
    # YOUR CODE STARTS HERE
    
    
    
    
    
    # YOUR CODE ENDS HERE
    
    # 2. Notch filter (60 Hz)
    # Usar scipy.signal.iirnotch y filtfilt
    # (approx. 3 lines)
    # YOUR CODE STARTS HERE
    
    
    
    # YOUR CODE ENDS HERE
    
    # 3. Common average reference
    # Restar el promedio de todos los canales
    # (approx. 2 lines)
    # YOUR CODE STARTS HERE
    
    
    # YOUR CODE ENDS HERE
    
    # 4. Normalización z-score por canal
    # Usar scipy.stats.zscore en cada canal
    # (approx. 3 lines)
    # YOUR CODE STARTS HERE
    
    
    
    # YOUR CODE ENDS HERE
    
    return eeg_normalized

In [ ]:
# Test Ejercicio 1
eeg_prep = preprocess_eeg(eeg_interictal)
print(f"✅ Preprocesado: {eeg_prep.shape}")
test_preprocess_eeg_tutorial6(preprocess_eeg)

<a name='5'></a>
## 5 - Extracción de Características Topológicas

[Volver al índice](#toc)

<a name='ex-2'></a>
### Ejercicio 2 - extract_comprehensive_features

Extrae características TDA + espectrales + temporales para ML.

**Incluir:**
- TDA: n_cycles, max_persistence, mean_persistence, entropy
- Espectrales: potencia por banda, frecuencia dominante
- Temporales: media, std, skewness, kurtosis

In [ ]:
# EJERCICIO 2: Extraer Características Completas

def extract_comprehensive_features(eeg_segment, fs=256):
    """
    Extrae características completas: TDA + espectrales + temporales.
    
    Arguments:
    eeg_segment -- array (n_channels, n_samples)
    fs -- frecuencia de muestreo
    
    Returns:
    features -- diccionario con todas las características
    """
    from scipy.fft import fft, fftfreq
    
    features = {}
    eeg_prep = preprocess_eeg(eeg_segment, fs)
    signal = eeg_prep[0]  # Usar primer canal
    
    # === TDA ===
    # Crear embedding de Takens simple
    delay = 10
    dim = 3
    embedded = np.column_stack([signal[i*delay:(len(signal)-(dim-1-i)*delay)] 
                                for i in range(dim)])
    
    # Subsampling para eficiencia
    if len(embedded) > 500:
        indices = np.random.choice(len(embedded), 500, replace=False)
        embedded = embedded[indices]
    
    # Calcular persistencia
    result = ripser(embedded, maxdim=1, thresh=5.0)
    dgm1 = result['dgms'][1]
    
    # Extraer características de H₁
    # (approx. 10 lines)
    # YOUR CODE STARTS HERE
    
    
    
    
    
    
    
    
    
    
    # YOUR CODE ENDS HERE
    
    # === ESPECTRALES ===
    # Calcular FFT y potencia por bandas
    # (approx. 8 lines)
    # YOUR CODE STARTS HERE
    
    
    
    
    
    
    
    
    # YOUR CODE ENDS HERE
    
    # === TEMPORALES ===
    # Estadísticas básicas de la señal
    # (approx. 5 lines)
    # YOUR CODE STARTS HERE
    
    
    
    
    
    # YOUR CODE ENDS HERE
    
    return features

In [ ]:
# Test Ejercicio 2
features_inter = extract_comprehensive_features(eeg_interictal)
features_ictal = extract_comprehensive_features(eeg_ictal)
print(f"✅ Features extraídas: {len(features_inter)} características")
print(f"   Interictal - ciclos: {features_inter.get('n_cycles', 0)}")
print(f"   Ictal - ciclos: {features_ictal.get('n_cycles', 0)}")
test_extract_comprehensive_features_tutorial6(extract_comprehensive_features)

<a name='6'></a>
## 6 - Machine Learning

[Volver al índice](#toc)

<a name='ex-3'></a>
### Ejercicio 3 - train_topological_classifier

Entrena clasificador usando características topológicas.

**Pasos:**
1. Dividir datos en train/test
2. Normalizar características
3. Entrenar Random Forest
4. Evaluar con métricas clínicas

In [ ]:
# EJERCICIO 3: Entrenar Clasificador

def train_topological_classifier(X, y, test_size=0.3):
    """
    Entrena y evalúa clasificador topológico.
    
    Arguments:
    X -- características (n_samples, n_features)
    y -- etiquetas (n_samples,)
    test_size -- proporción de test
    
    Returns:
    clf -- clasificador entrenado
    results -- diccionario con métricas
    """
    from sklearn.preprocessing import StandardScaler
    
    results = {}
    
    # 1. Dividir datos
    # (approx. 2 lines)
    # YOUR CODE STARTS HERE
    
    
    # YOUR CODE ENDS HERE
    
    # 2. Normalizar
    # (approx. 3 lines)
    # YOUR CODE STARTS HERE
    
    
    
    # YOUR CODE ENDS HERE
    
    # 3. Entrenar Random Forest
    # (approx. 2 lines)
    # YOUR CODE STARTS HERE
    
    
    # YOUR CODE ENDS HERE
    
    # 4. Predicciones y métricas
    # (approx. 5 lines)
    # YOUR CODE STARTS HERE
    
    
    
    
    
    # YOUR CODE ENDS HERE
    
    return clf, results

In [ ]:
# Generar dataset completo
print("🏗️ Construyendo dataset...\n")
X_data = []
y_data = []

for i in range(30):
    eeg, _ = generate_realistic_eeg_segment(duration=10, state='interictal')
    features = extract_comprehensive_features(eeg)
    X_data.append(list(features.values()))
    y_data.append(0)

for i in range(30):
    eeg, _ = generate_realistic_eeg_segment(duration=10, state='ictal')
    features = extract_comprehensive_features(eeg)
    X_data.append(list(features.values()))
    y_data.append(1)

X = np.array(X_data)
y = np.array(y_data)
print(f"✅ Dataset: {X.shape}")

In [ ]:
# Test Ejercicio 3
clf, results = train_topological_classifier(X, y)
print(f"\n✅ Accuracy: {results.get('accuracy', 0):.2%}")
test_train_topological_classifier(train_topological_classifier)

<a name='7'></a>
## 7 - Resumen

[Volver al índice](#toc)

<div style="background-color:#e3f2fd; padding:15px; border-left:5px solid #2196f3;">

**💡 Lo que aprendimos:**

- **Preprocesamiento profesional** es crítico
- **TDA** aporta características únicas
- **Pipeline completo** funciona en datos reales
- **Clasificación** con alta precisión es posible
- **Aplicación clínica** promete mejorar diagnóstico

</div>

---

## 🎉 ¡Felicitaciones!

Completaste el caso de estudio end-to-end. Ahora dominas el pipeline completo de TDA aplicado.

---

**Autor:** MARK-126  
**Licencia:** MIT

---

### Ejercicio 4 - feature_importance_analysis

Entender qué **features topológicas** son más discriminativas para detectar epilepsia permite interpretabilidad clínica y optimización del modelo. Features como Betti numbers específicos o rangos de persistencia pueden tener significado neurobiológico.

**Aplicación clínica:** Identificar biomarcadores interpretables, optimizar panel de features.

**Tu tarea:** Implementa un análisis completo de importancia de features usando múltiples métodos.

**Dificultad:** ⭐⭐⭐ Avanzado
**Tiempo estimado:** 20-25 minutos

In [ ]:
def feature_importance_analysis():
    """
    Analiza importancia de features topológicas para clasificación.

    Parámetros:
    -----------
    X : array, shape (n_samples, n_features)
        Matriz de features TDA
    y : array, shape (n_samples,)
        Labels (0=normal, 1=ictal)
    feature_names : list
        Nombres de las features

    Retorna:
    --------
    importance_scores : dict
        Diccionario con scores de diferentes métodos:
        - 'random_forest': Importancia de Random Forest
        - 'permutation': Importancia por permutación
        - 'mutual_info': Información mutua
    top_features : list
        Top 10 features más importantes (nombres)
    """
    # YOUR CODE STARTS HERE
    # (approx. 20-25 lines)
    # Hint 1: Entrena RandomForestClassifier y extrae feature_importances_
    # Hint 2: Calcula permutation importance con sklearn.inspection
    # Hint 3: Calcula mutual information con sklearn.feature_selection
    # Hint 4: Normaliza todos los scores a [0, 1]
    # Hint 5: Combina scores (promedio o ranking fusion)
    # Hint 6: Retorna top 10 features por importancia combinada

    # YOUR CODE ENDS HERE

In [ ]:
# Test automático
from notebooks.tda_tests import test_feature_importance_analysis
test_feature_importance_analysis(feature_importance_analysis)

---

### Ejercicio 5 - cross_validate_pipeline

La **validación cruzada rigurosa** es esencial para estimar el desempeño real del sistema en datos nuevos. Usar estratificación y validación temporal evita sobreajuste y produce estimaciones realistas.

**Aplicación:** Validación pre-clínica antes de deployment, reporte de desempeño confiable.

**Tu tarea:** Implementa un pipeline completo de validación cruzada con múltiples métricas.

**Dificultad:** ⭐⭐⭐ Avanzado
**Tiempo estimado:** 20-25 minutos

In [ ]:
def cross_validate_pipeline():
    """
    Valida pipeline completo de detección usando cross-validation.

    Parámetros:
    -----------
    eeg_data : array, shape (n_epochs, n_channels, n_samples)
        Datos EEG crudos
    labels : array, shape (n_epochs,)
        Labels verdaderas
    cv_folds : int
        Número de folds para cross-validation (default: 5)

    Retorna:
    --------
    cv_results : dict
        Resultados de validación cruzada:
        - 'accuracy': Accuracy promedio ± std
        - 'precision': Precision promedio ± std
        - 'recall': Recall promedio ± std
        - 'f1': F1-score promedio ± std
        - 'roc_auc': ROC-AUC promedio ± std
        - 'confusion_matrices': Lista de matrices de confusión
    trained_model : objeto
        Modelo final entrenado en todos los datos
    """
    # YOUR CODE STARTS HERE
    # (approx. 25-30 lines)
    # Hint 1: Preprocesa todos los epochs de EEG (filtros, CAR, normalización)
    # Hint 2: Extrae features TDA + espectrales de cada epoch
    # Hint 3: Usa StratifiedKFold para splits balanceados
    # Hint 4: Para cada fold:
    #         - Entrena pipeline en train set
    #         - Evalúa en validation set
    #         - Calcula todas las métricas
    # Hint 5: Agrega resultados y calcula media ± std
    # Hint 6: Entrena modelo final en todos los datos
    # Hint 7: Retorna resultados completos de CV

    # YOUR CODE ENDS HERE

In [ ]:
# Test automático
from notebooks.tda_tests import test_cross_validate_pipeline
test_cross_validate_pipeline(cross_validate_pipeline)